<a href="https://colab.research.google.com/github/GaCampioto/tech-challenges-fiap-AI-for-developers/blob/main/tech_challenge_3_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tech Challenge 3 - Preparação do dataset

O arquivo disponibilizado para realização da tarefa foi o [AmazonTitles1.3MM](https://drive.google.com/file/d/12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK/view)

Dentro do arquivo é possível encontrar os seguintes arquivos:

- filter_labels_test.txt
- filter_labels_train.txt
- lbl.json.gz
- trn.json.gz
- lbl.json.gz

Para este tech challenge vamos usar somente o trn.json.gz
Este notebook foi executado em uma T4 RAM alta.

In [1]:
# habilitando a utilização dos arquivos do drive como fonte de dados
from google.colab import drive
drive.mount('/content/drive')

tc3_path = '/content/drive/MyDrive/Colab Notebooks/tech-challenge-3/'

Mounted at /content/drive


## Conhecendo os dados

Após verificar cada um dos arquivos, acredito que usaremos somente o `trn.json.gz`, `tst.json.gz` e o `lbl.json.gz` sendo respectivamente responsáveis pelos dados de treino (features e targets), teste (features e targets) e validação do modelo.

Para iniciar, vamos carregar cada um dos arquivos para um dataframe

In [2]:
import pandas, gzip

trn_df = pandas.read_json(gzip.open(f'{tc3_path}/trn.json.gz'), lines=True)

In [ ]:
trn_df.head()

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,0000032034,Adult Ballet Tutu Yellow,,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,0000913154,The Way Things Work: An Illustrated Encycloped...,,"[116, 117, 118, 119, 120, 121, 122]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
4,0001381245,Misty of Chincoteague,,[151],[1.0]


In [ ]:
(trn_df['title'].values == '').sum()

126834

In [ ]:
(trn_df['content'].values == '').sum()

749901

In [ ]:
trn_df.shape

(2248619, 5)

Neste ponto conseguimos concluir que o dataframe `trn_df` possui alguns valores vazios nas colunas de `title` e `content`.

## Limpeza e pré processamento

Como o objetivo do tech challenge é realizar o fine tuning com todo o arquivo `trn.json.gz` utilizando o `title` e o `content` os outros atributos  (`uid, target_ind, target_rel)` e registros com valores vazios serão desconsiderados (title = 126834, content = 749901)


In [3]:
import numpy as np

trn_df.drop(columns=['uid', 'target_ind', 'target_rel'], inplace=True)
print(trn_df.shape)
trn_df['title'].replace('', np.nan, inplace=True)
trn_df['content'].replace('', np.nan, inplace=True)
trn_df.dropna(subset=['title'], inplace=True)
trn_df.dropna(subset=['content'], inplace=True)
print(trn_df.shape)
trn_df

trn_df.head()

(2248619, 2)
(1390403, 2)


,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
3,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...
7,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
12,The Prophet,"In a distant, timeless place, a mysterious pro..."
13,Rightly Dividing the Word,--This text refers to thePaperbackedition.


Agora com os dados organizados, vamos montar nossos `trn_prepared_index.json`, incluindo no `input` o prompt que usaremos para o treinamento do modelo.

In [4]:
import json

def default(obj):
  if isinstance(obj, np.ndarray):
    return obj.tolist()
  elif isinstance(obj, np.integer):
    return int(obj)
  else:
    return super().default(obj)

def prepare_data(df):
  data = []
  for index, row in df.iterrows():
    data.append({"input":  f"Write a content for the title above.\n[|Title|]{row['title']}[|eTitle|]\n\n[|content|]{row['content']}[|eContent|]"})
  return data

def export(path, data):
  with open(path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4, default=default)

In [ ]:
import numpy as np

data = prepare_data(trn_df)
chunked_data = np.array_split(data, 10)
index = 1
for chunk in chunked_data:
  export(f'{tc3_path}/trn_prepared_{index}.json', chunk)
  index += 1